# Ataca la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Tendríamos que generar un main.py (más los scripts que se te ocurran) que guarde un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [1]:
import pandas as pd
import marvel as mv
from api_keys import api_private_key_marvel, api_public_key_marvel
import requests
import hashlib
import time
import csv


pd.set_option("max_colwidth", 100)
# pd.set_option("max_rows", 1000)

In [2]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [9]:


def generate_hash(ts, private_key, public_key):
    # Esta función genera el hash MD5 necesario para la autenticación con la API de Marvel
    string_to_hash = f"{ts}{private_key}{public_key}"
    return hashlib.md5(string_to_hash.encode('utf-8')).hexdigest()

def get_marvel_characters(starting_letter):
    # Esta función realiza la solicitud a la API de Marvel y devuelve la información de los personajes
    url = 'https://gateway.marvel.com/v1/public/characters'
    public_key = api_public_key_marvel  # Reemplaza con tu clave pública de Marvel
    private_key = api_private_key_marvel  # Reemplaza con tu clave privada de Marvel
    ts = str(int(time.time()))  # Se obtiene la marca de tiempo actual en segundos
    hash_value = generate_hash(ts, private_key, public_key)
    offset = 0
    limit = 100  # Puedes ajustar el límite según tus necesidades

    all_characters = []

    while True:

        params = {
            'apikey': public_key,
            'ts': ts,
            'hash': hash_value,
            'nameStartsWith': starting_letter,
            'offset': offset,
            'limit': limit,
        }

        # Realiza la solicitud GET a la API de Marvel con los parámetros especificados
        response = requests.get(url, params=params)

        if response.status_code == 200:
            # Si la respuesta es exitosa (código de estado 200), se obtiene y devuelve la información de los personajes
            data = response.json()
            results = data.get('data', {}).get('results', [])
            if not results:
                break  # Si no hay más resultados, sal del bucle
            all_characters.extend(results)
            offset += limit
        else:
            # Si hay un error en la respuesta, se imprime un mensaje de error y se devuelve una lista vacía
            print(f'Error al obtener datos. Código de estado: {response.status_code}')
            break
    return all_characters

def save_to_csv(characters):
    # Esta función guarda la información de los personajes en un archivo CSV
    with open('marvel_characters.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'name', 'picture_url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Escribe la fila de encabezado en el archivo CSV
        
        for character in characters:
            # Escribe cada personaje como una fila en el archivo CSV
            writer.writerow({
                'id': character['id'],
                'name': character['name'],
                'picture_url': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}",
            })

def main():
    # Función principal que se ejecuta al correr el script
    starting_letter = 'L'  # Cambia la letra según tu preferencia
    characters = get_marvel_characters(starting_letter)

    if characters:
        # Si se obtienen personajes, se guarda la información en un archivo CSV
        save_to_csv(characters)
        print('Datos guardados correctamente en marvel_characters.csv')
    else:
        # Si no se obtienen personajes, se imprime un mensaje de advertencia
        print('No se encontraron personajes de Marvel con la letra proporcionada.')

if __name__ == "__main__":
    # Esta condición verifica si el script se ejecuta directamente y no se importa como módulo
    main()


Datos guardados correctamente en marvel_characters.csv


In [10]:
# Lee el archivo CSV y carga los datos en un DataFrame
df = pd.read_csv('marvel_characters.csv')

# Muestra las primeras filas del DataFrame para verificar
df

,id,name,picture_url
0,1009392,La Nuit,http://i.annihil.us/u/prod/marvel/i/mg/9/10/4c003d76b5ec6.jpg
1,1011327,Lady Bullseye,http://i.annihil.us/u/prod/marvel/i/mg/9/90/4ce5a67d44f61.jpg
2,1009393,Lady Deathstrike,http://i.annihil.us/u/prod/marvel/i/mg/9/60/535ff2c1ef191.jpg
3,1011014,Lady Mastermind,http://i.annihil.us/u/prod/marvel/i/mg/5/d0/4c00326f63d4c.jpg
4,1014977,Lady Ursula,http://i.annihil.us/u/prod/marvel/i/mg/c/40/4cd053ea971ed.jpg
5,1014976,Lady Vermin,http://i.annihil.us/u/prod/marvel/i/mg/6/10/4cd053529dd41.jpg
6,1009394,Lake,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
7,1009395,Landau,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
8,1009397,Lava-Man,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
9,1011100,Layla Miller,http://i.annihil.us/u/prod/marvel/i/mg/3/00/52740e37c104f.jpg


,id,name,picture_url
0,1009637.0,Sunset Bain,http://i.annihil.us/u/prod/marvel/i/mg/c/10/4c003ab9ed7d6.jpg
1,1009638.0,Sunspot,http://i.annihil.us/u/prod/marvel/i/mg/b/50/4c003ab716add.jpg
2,1015280.0,Super Hero Squad,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1011022.0,Super-Adaptoid,http://i.annihil.us/u/prod/marvel/i/mg/5/e0/4c00325af279c.jpg
4,1009639.0,Super-Skrull,http://i.annihil.us/u/prod/marvel/i/mg/9/b0/5274138fe6deb.jpg
...,...,...,...
257,1011515.0,Zeus,http://i.annihil.us/u/prod/marvel/i/mg/f/60/4ce5a7fcaa386.png
258,1011127.0,Zodiak,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
259,1009741.0,Zombie (Simon Garth),http://i.annihil.us/u/prod/marvel/i/mg/6/10/4c003937c9ba4.jpg
260,1011183.0,Zuras,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg


,id,name,picture_url
0,1011334.0,3-D Man,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784.jpg
1,1017100.0,A-Bomb (HAS),http://i.annihil.us/u/prod/marvel/i/mg/3/20/5232158de5b16.jpg
2,1009144.0,A.I.M.,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52602f21f29ec.jpg
3,1010699.0,Aaron Stack,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
4,1009146.0,Abomination (Emil Blonsky),http://i.annihil.us/u/prod/marvel/i/mg/9/50/4ce18691cbf04.jpg
...,...,...,...
1557,1011515.0,Zeus,http://i.annihil.us/u/prod/marvel/i/mg/f/60/4ce5a7fcaa386.png
1558,1011127.0,Zodiak,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
1559,1009741.0,Zombie (Simon Garth),http://i.annihil.us/u/prod/marvel/i/mg/6/10/4c003937c9ba4.jpg
1560,1011183.0,Zuras,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg


## Encapsúlalo en un main.py, funciones.py y variables.py